In [21]:
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [20]:
#READ

#read data
df = pd.read_csv("data.csv")

#select data
selected = ["Weapon", "Range", "DropOff", 
            "Zero Damage Range", "RPM","Capacity", "ReloadSpeed(ms)","ReloadSpeedFromEmpty(ms)", 
            "HeadshotMultiplier", "WeaponBonus", "MaxBonus%", "regularDMG", "minimumDMG"]
df = df[selected]

#CLEAN

#replacement
df["WeaponBonus"] = df["WeaponBonus"].str.replace("-", "No Bonus").astype(str)

df['MaxBonus%'] = df['MaxBonus%'].str.replace('%', '').astype(str)
df['MaxBonus%'] = df['MaxBonus%'].str.replace('-', '0').astype(str)

df['HeadshotMultiplier'] = df['HeadshotMultiplier'].str.replace('1 \+ Bonus', '1',regex=True).astype(str)
df['HeadshotMultiplier'] = df['HeadshotMultiplier'].str.replace('1\+Bonus', '1',regex=True).astype(str) #this also happens
#type setting. (Just in case it needs to be done)


In [ ]:
df

In [23]:
weights = {
    "regularDMG": 0.4,
    
}